In [107]:
# Constants
ALPH = "абвгдежзийклмнопрстуфхцчшщъыьэюя"
PROBABILITIES = [0.0792, 0.0171, 0.0433, 0.0174, 0.0305, 0.0841, 0.0105, 0.0175,
                 0.0683, 0.0112, 0.0336, 0.0500, 0.0326, 0.0672, 0.1108, 0.0281,
                 0.0445, 0.0533, 0.0618, 0.0280, 0.0019, 0.0089, 0.0036, 0.0147,
                 0.0081, 0.0037, 0.0002, 0.0196, 0.0192, 0.0038, 0.0061, 0.0213]
ALPH_SIZE = len(ALPH)
RING = {}
for char in ALPH:
    pos = ALPH.index(char)
    RING[char] = pos
EXPECTED_I = sum([(p ** 2) for p in PROBABILITIES])
I0 = 1.0 / ALPH_SIZE
FILE_NAME = "text.txt"

EXPECTED_I

0.055300129999999996

In [108]:
def transform_symbol(_c):
    if 'а' <= _c and _c <= 'я':
        return _c
    elif _c <= 'Я' and _c >= 'А':
        return _c.lower()
    elif _c == 'Ё' or _c == 'ё':
        return 'е'
    else:
        return ""
    
def preprocess_text(_text):
    text_formatted = ""
    # Change symbols according to requirements
    for c in _text:
        text_formatted += transform_symbol(c)
# Треба функція для додавання 
    # Remove consequtive spaces
    text_formatted = ' '.join(text_formatted.split())
    return text_formatted

def read_text(filename):
    f = open(filename, "r", encoding='utf-8')
    text = f.read()
    f.close()
    return preprocess_text(text)

In [109]:
def transform_symbol(_c):
    if 'а' <= _c and _c <= 'я':
        return _c
    elif _c <= 'Я' and _c >= 'А':
        return _c.lower()
    elif _c == 'Ё' or _c == 'ё': 
        return 'е'
    else:
        return ""
    
def preprocess_text(_text):
    text_formatted = ""
    # Change symbols according to requirements
    for c in _text:
        text_formatted += transform_symbol(c)
    # Remove consequtive spaces
    text_formatted = ' '.join(text_formatted.split())
    return text_formatted

def read_text(filename):
    f = open(filename, "r", encoding='utf-8')
    text = f.read()
    f.close()
    return preprocess_text(text)

In [110]:
def coincidence_index(text: list) -> int:
    sum = 0
    for c in ALPH:
        occurances = text.count(c)
        sum += occurances * (occurances - 1)
    return sum / (len(text) * (len(text) - 1))

def divide_into_blocks(text: list, r: int):
    res = []
    for i in range(r):
        block = text[i::r] 
        res.append(block)
    # print(res)
    return res

def coinc_idx_for_blocks(text: list, r: int):
    idxs = []
    blocks = divide_into_blocks(text, r)
    for i in range(r):
        idxs.append(coincidence_index(blocks[i]))
    return idxs

def summary_af_idx_for_diff_r(text: list):
    idx_table = {}
    # if we assume that len(KEY) divides len(text) -> itr through divisors
    itr_range = range(2, int(len(text) / 2))     
    for r in itr_range:
        idx_table[r] = coinc_idx_for_blocks(text, r)
    return idx_table

In [111]:
import numpy.linalg as linalg
# Instead of comparing average idx diff, we measure vector distance and compare
def compare_idx_vectors(left: list, right: list) -> bool:
    left_dist = linalg.norm([EXPECTED_I - idx for idx in left])
    right_dist = linalg.norm([EXPECTED_I - idx for idx in right])
    return left_dist <= right_dist

def get_best_r(text: list):
    best_r = 2
    idx_map = summary_af_idx_for_diff_r(text)
    for (r, idx) in idx_map.items():
        if compare_idx_vectors(idx, idx_map[best_r]):
            best_r = r
            print(f"{r}: {sum(idx) / len(idx)}")
        # if abs(EXPECTED_I - idx) <= abs(EXPECTED_I - idx_map[best_r]):
        #     best_r = r
        #     print(f"{r}: {idx}")
    return best_r


In [ ]:
# def crack_key(text, r) -> str:
#     key = []
#     blocks = divide_into_blocks(text, r)
#     for i in range(0, r-1):
#         ki = []
#         for k in range(0, ALPH_SIZE-1):
#              for c in ALPH:
#                 occurances = text.count(c)
#                 sum += occurances * (occurances - 1)


In [112]:
text = read_text(FILE_NAME)
get_best_r(text)
# len(text)
# [EXPECTED_I] * 10

2: 0.03626820548217928
14: 0.05528168514213951


14